In [1]:
from headfile import *

In [2]:
# 读
df = pd.read_excel("数据预处理.xlsx",parse_dates=["order_date"])
df

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty
0,2015-09-01,104,22069,307,403,offline,1114.00,19
1,2015-09-01,104,20028,301,405,offline,1012.00,12
2,2015-09-02,104,21183,307,403,online,428.00,109
3,2015-09-02,104,20448,308,404,online,962.00,3
4,2015-09-02,104,21565,307,403,offline,1400.00,3
...,...,...,...,...,...,...,...,...
597688,2018-12-20,102,20994,302,408,offline,843.00,59
597689,2018-12-20,102,21875,302,408,offline,762.00,502
597690,2018-12-20,102,20215,302,408,offline,2013.00,106
597691,2018-12-20,102,20195,302,408,offline,2120.00,187


In [3]:
# 天
#
# freq = df['sales_chan_name'].value_counts()/len(df['sales_chan_name'])
# df['offline_sales_freq'] = df['sales_chan_name'].apply(lambda x:freq[x])
# df=df.drop(["sales_chan_name"],axis=1)
# 新增属于哪一年属性
df['which_year'] = df['order_date'].dt.year

# 新增一年第几天属性
df['day_of_year'] = df['order_date'].dt.dayofyear

# 新增本周星期几属性（0代表星期一，6代表星期天）
df['weekday'] = df['order_date'].dt.weekday

# 新增本月第几天属性
df['day_of_month'] = df['order_date'].dt.day

# 新增本月第几天属性
df['day_of_month'] = df['order_date'].dt.day

# 新增是否属于促销日属性（这里以618, 双11和双12为例）
promotions = []
for year in range(df['order_date'].dt.year.min(), df['order_date'].dt.year.max()+1):
    promotions += [pd.Timestamp(year=year, month=6, day=18), pd.Timestamp(year=year, month=11, day=11), pd.Timestamp(year=year, month=12, day=12)]
df['is_promotion_day'] = np.where(df['order_date'].isin(promotions), 1, 0)

# 新增是否属于节假日属性（这里以中国公共节假日为例）
df['is_holiday'] = df['order_date'].apply(lambda x: 1 if is_holiday(x) else 0)


# 新增季节属性
# seasons = ['winter', 'spring', 'summer', 'fall']
seasons = ['0', '1', '2', '3']
month_to_season = dict(zip(range(1,13), ['0']*2+['1']*3+['2']*3+['3']*3+['0']))
df['season'] = df['order_date'].dt.month.map(month_to_season)

# 新增本季节第几天属性
def day_of_season(date):
    year_start = pd.Timestamp(year=date.year, month=1, day=1)
    season_start = pd.Timestamp(year=date.year, month=(date.month-1)//3*3+1, day=1)
    return (date - season_start).days + 1

df['day_of_season'] = df['order_date'].apply(day_of_season)
df['sales_chan_name']= df['sales_chan_name'].apply(lambda x: 1 if x == 'offline' else 0)
# 打印结果
df.head()

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,which_year,day_of_year,weekday,day_of_month,is_promotion_day,is_holiday,season,day_of_season
0,2015-09-01,104,22069,307,403,1,1114.00,19,2015,244,1,1,0,0,3,63
1,2015-09-01,104,20028,301,405,1,1012.00,12,2015,244,1,1,0,0,3,63
2,2015-09-02,104,21183,307,403,0,428.00,109,2015,245,2,2,0,0,3,64
3,2015-09-02,104,20448,308,404,0,962.00,3,2015,245,2,2,0,0,3,64
4,2015-09-02,104,21565,307,403,1,1400.00,3,2015,245,2,2,0,0,3,64


In [4]:
df.describe()

,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,which_year,day_of_year,weekday,day_of_month,is_promotion_day,is_holiday,day_of_season
count,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00
mean,102.90,21008.02,304.93,406.46,0.74,1076.24,91.65,2017.01,200.67,2.97,16.90,0.01,0.26,48.28
std,1.50,590.90,2.00,3.11,0.44,1167.51,199.84,0.92,106.84,1.90,8.40,0.09,0.44,26.13
min,101.00,20001.00,301.00,401.00,0.00,1.00,1.00,2015.00,1.00,0.00,1.00,0.00,0.00,1.00
25%,102.00,20518.00,303.00,404.00,0.00,598.00,10.00,2016.00,107.00,1.00,10.00,0.00,0.00,25.00
50%,103.00,20976.00,306.00,407.00,1.00,883.00,29.00,2017.00,218.00,3.00,17.00,0.00,0.00,49.00
75%,105.00,21521.00,306.00,408.00,1.00,1291.00,101.00,2018.00,296.00,5.00,24.00,0.00,1.00,72.00
max,105.00,22084.00,308.00,412.00,1.00,260014.00,16308.00,2018.00,366.00,6.00,31.00,1.00,1.00,92.00


In [5]:
df['order_date']=df['order_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
ywhfun.save(df, "特征衍生_天.xlsx", type_file="xlsx")

In [6]:
# 读-周
df = pd.read_excel("数据预处理.xlsx",parse_dates=["order_date"])
df

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty
0,2015-09-01,104,22069,307,403,offline,1114.00,19
1,2015-09-01,104,20028,301,405,offline,1012.00,12
2,2015-09-02,104,21183,307,403,online,428.00,109
3,2015-09-02,104,20448,308,404,online,962.00,3
4,2015-09-02,104,21565,307,403,offline,1400.00,3
...,...,...,...,...,...,...,...,...
597688,2018-12-20,102,20994,302,408,offline,843.00,59
597689,2018-12-20,102,21875,302,408,offline,762.00,502
597690,2018-12-20,102,20215,302,408,offline,2013.00,106
597691,2018-12-20,102,20195,302,408,offline,2120.00,187


In [7]:
# 获取该月第几周、促销日和节假日数量、季节以及本月和本季节第几周
def get_week_month_season_prom_hol(df):
    df['year'] = df['order_date'].apply(lambda x: x.year)
    df['season'] = df['order_date'].apply(lambda x: (x.month - 1) // 3 + 1)
    df['month'] = df['order_date'].dt.month
    df['week'] = df['order_date'].dt.week
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['month_week'] = df['order_date'].apply(lambda x: (x.day - 1) // 7 + 1)
    df = df.sort_values(by=['sales_region_code', 'item_code', 'order_date']).reset_index(drop=True)
    df['season_week'] = df.groupby(['sales_region_code', 'item_code', 'month'])['order_date'].apply(lambda x: (x.rank(method='dense') - 1) // 7 + 1)
    df['promotion_days'] = df['order_date'].apply(lambda x: 1 if (x.month == 6 and x.day == 18) or (x.month == 11 and x.day in [11, 12]) else 0)
    df['holiday_days'] = df['order_date'].apply(lambda x: 1 if is_holiday(x) else 0)
    
    return df

# 计算属性
df_data = get_week_month_season_prom_hol(df)
df_data

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,year,season,month,week,month_week,season_week,promotion_days,holiday_days
0,2016-03-15,101,20001,302,408,offline,700.00,102,2016,1,3,11,3,1.00,0,0
1,2016-03-21,101,20001,302,408,offline,705.00,19,2016,1,3,12,3,1.00,0,0
2,2016-03-23,101,20001,302,408,offline,702.00,36,2016,1,3,12,4,1.00,0,0
3,2016-03-24,101,20001,302,408,offline,692.00,204,2016,1,3,12,4,1.00,0,0
4,2016-03-25,101,20001,302,408,offline,693.00,36,2016,1,3,12,4,1.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597688,2018-08-25,105,22084,302,408,offline,1038.00,10,2018,3,8,34,4,1.00,0,1
597689,2018-09-12,105,22084,302,408,offline,659.00,106,2018,3,9,37,2,1.00,0,0
597690,2018-09-22,105,22084,302,408,offline,1034.00,5,2018,3,9,38,4,1.00,0,1
597691,2018-09-30,105,22084,302,408,offline,1041.00,11,2018,3,9,39,5,1.00,0,0


In [8]:
df_data.describe()

,sales_region_code,item_code,first_cate_code,second_cate_code,item_price,ord_qty,year,season,month,week,month_week,season_week,promotion_days,holiday_days
count,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00
mean,102.90,21008.02,304.93,406.46,1076.24,91.65,2017.01,2.68,7.06,29.03,2.86,2.65,0.01,0.26
std,1.50,590.90,2.00,3.11,1167.51,199.84,0.92,1.14,3.50,15.30,1.23,2.11,0.09,0.44
min,101.00,20001.00,301.00,401.00,1.00,1.00,2015.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00
25%,102.00,20518.00,303.00,404.00,598.00,10.00,2016.00,2.00,4.00,16.00,2.00,1.00,0.00,0.00
50%,103.00,20976.00,306.00,407.00,883.00,29.00,2017.00,3.00,8.00,32.00,3.00,2.00,0.00,0.00
75%,105.00,21521.00,306.00,408.00,1291.00,101.00,2018.00,4.00,10.00,43.00,4.00,3.00,0.00,1.00
max,105.00,22084.00,308.00,412.00,260014.00,16308.00,2018.00,4.00,12.00,53.00,5.00,16.00,1.00,1.00


In [9]:
# 统计与合并
group_cols = ['year', 'week', 'item_code']
agg_dict = {
    'sales_region_code': 'first',
    'first_cate_code': 'mean',
    'second_cate_code': 'mean',
    'sales_chan_name': lambda x: (x == 'offline').sum()/len(x),
    'item_price': 'mean',
    'ord_qty': 'sum',
    'promotion_days': 'max',
    'holiday_days': 'max',
    'season': 'max',
    'season_week': 'max',
    'month_week': 'max'
}
df_result = df_data.groupby(group_cols).agg(agg_dict).reset_index()
df_result.insert(0,'order_date',df_data['order_date'] - pd.to_timedelta(df_data['order_date'].dt.dayofweek, unit='D'))
df_result['order_date']=df_result['order_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_result = df_result.rename(columns={
    'item_price': 'mean_item_price',
    'ord_qty': 'sum_ord_qty'
}).sort_values(group_cols + ['sales_region_code'])
df_result

,order_date,year,week,item_code,sales_region_code,first_cate_code,second_cate_code,sales_chan_name,mean_item_price,sum_ord_qty,promotion_days,holiday_days,season,season_week,month_week
0,2016-03-14,2015,36,20003,104,301,405,0.00,740.00,83,0,1,3,1.00,1
1,2016-03-21,2015,36,20007,101,305,412,0.67,100.33,1165,0,0,3,1.00,1
2,2016-03-21,2015,36,20008,102,303,401,1.00,658.75,172,0,1,3,1.00,1
3,2016-03-21,2015,36,20009,105,306,407,0.00,678.00,188,0,0,3,1.00,1
4,2016-03-21,2015,36,20020,101,305,412,1.00,1912.67,48,0,1,3,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76031,2018-06-11,2018,51,22072,105,305,412,0.00,427.00,248,0,0,4,1.00,3
76032,2018-06-18,2018,51,22081,102,303,401,1.00,3805.60,49,0,0,4,3.00,3
76033,2018-06-18,2018,51,22082,102,305,412,1.00,2057.50,18,0,0,4,2.00,3
76034,2018-07-23,2018,51,22083,103,303,401,0.00,1715.00,62,0,0,4,2.00,3


In [10]:
df_result.describe()

,year,week,item_code,sales_region_code,first_cate_code,second_cate_code,sales_chan_name,mean_item_price,sum_ord_qty,promotion_days,holiday_days,season,season_week,month_week
count,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00,76036.00
mean,2016.91,28.98,21020.17,102.33,304.73,406.74,0.70,1210.95,720.43,0.02,0.63,2.68,2.37,3.21
std,0.96,15.01,603.82,1.57,2.03,3.21,0.41,2102.90,1800.22,0.15,0.48,1.11,1.88,1.24
min,2015.00,1.00,20001.00,101.00,301.00,401.00,0.00,2.01,1.00,0.00,0.00,1.00,1.00,1.00
25%,2016.00,16.00,20504.00,101.00,303.00,404.00,0.29,630.00,39.00,0.00,0.00,2.00,1.00,2.00
50%,2017.00,30.00,20989.00,102.00,305.00,407.00,1.00,942.11,177.00,0.00,1.00,3.00,2.00,3.00
75%,2018.00,42.00,21549.00,104.00,306.00,408.00,1.00,1370.58,641.00,0.00,1.00,4.00,3.00,4.00
max,2018.00,53.00,22084.00,105.00,308.00,412.00,1.00,130491.50,56355.00,1.00,1.00,4.00,16.00,5.00


In [11]:
ywhfun.save(df_result, "特征衍生_周.xlsx", type_file="xlsx")

In [12]:
# 读-月
df_month = pd.read_excel("数据预处理.xlsx",parse_dates=["order_date"])
df_month

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty
0,2015-09-01,104,22069,307,403,offline,1114.00,19
1,2015-09-01,104,20028,301,405,offline,1012.00,12
2,2015-09-02,104,21183,307,403,online,428.00,109
3,2015-09-02,104,20448,308,404,online,962.00,3
4,2015-09-02,104,21565,307,403,offline,1400.00,3
...,...,...,...,...,...,...,...,...
597688,2018-12-20,102,20994,302,408,offline,843.00,59
597689,2018-12-20,102,21875,302,408,offline,762.00,502
597690,2018-12-20,102,20215,302,408,offline,2013.00,106
597691,2018-12-20,102,20195,302,408,offline,2120.00,187


In [13]:
# 获取本年的第几个月、季节、本季节的第几个月、促销日和节假日数量
def get_year_month_season_promotion_holiday(df):
    df['year'] = df['order_date'].apply(lambda x: x.year)
    df['month'] = df['order_date'].dt.month
    df['quarter_start_month'] = df['order_date'].apply(lambda x: (x.month - 1) // 3 * 3 + 1)
    df['season'] = df['order_date'].apply(lambda x: (x.month - 1) // 3 + 1)
    df['season_month'] = df.groupby(['sales_region_code', 'item_code', 'season'])['month'].transform(lambda x: x.rank(method='dense'))
    df['promotion'] = df['month'].apply(lambda x: 1 if x in [6, 11, 12] else 0)
    df['holiday'] = df['order_date'].apply(lambda x: 1 if is_holiday(x) else 0)
    return df
# 计算属性
df_data_month = get_year_month_season_promotion_holiday(df_month)
df_data_month

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,year,month,quarter_start_month,season,season_month,promotion,holiday
0,2015-09-01,104,22069,307,403,offline,1114.00,19,2015,9,7,3,1,0,0
1,2015-09-01,104,20028,301,405,offline,1012.00,12,2015,9,7,3,1,0,0
2,2015-09-02,104,21183,307,403,online,428.00,109,2015,9,7,3,3,0,0
3,2015-09-02,104,20448,308,404,online,962.00,3,2015,9,7,3,3,0,0
4,2015-09-02,104,21565,307,403,offline,1400.00,3,2015,9,7,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597688,2018-12-20,102,20994,302,408,offline,843.00,59,2018,12,10,4,2,1,0
597689,2018-12-20,102,21875,302,408,offline,762.00,502,2018,12,10,4,2,1,0
597690,2018-12-20,102,20215,302,408,offline,2013.00,106,2018,12,10,4,2,1,0
597691,2018-12-20,102,20195,302,408,offline,2120.00,187,2018,12,10,4,2,1,0


In [14]:
df_data_month.describe()

,sales_region_code,item_code,first_cate_code,second_cate_code,item_price,ord_qty,year,month,quarter_start_month,season,season_month,promotion,holiday
count,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00,597693.00
mean,102.90,21008.02,304.93,406.46,1076.24,91.65,2017.01,7.06,6.03,2.68,1.98,0.28,0.26
std,1.50,590.90,2.00,3.11,1167.51,199.84,0.92,3.50,3.41,1.14,0.82,0.45,0.44
min,101.00,20001.00,301.00,401.00,1.00,1.00,2015.00,1.00,1.00,1.00,1.00,0.00,0.00
25%,102.00,20518.00,303.00,404.00,598.00,10.00,2016.00,4.00,4.00,2.00,1.00,0.00,0.00
50%,103.00,20976.00,306.00,407.00,883.00,29.00,2017.00,8.00,7.00,3.00,2.00,0.00,0.00
75%,105.00,21521.00,306.00,408.00,1291.00,101.00,2018.00,10.00,10.00,4.00,3.00,1.00,1.00
max,105.00,22084.00,308.00,412.00,260014.00,16308.00,2018.00,12.00,10.00,4.00,3.00,1.00,1.00


In [15]:
# 统计合并
group_cols = ['year', 'month', 'item_code']
agg_dict = {
    'sales_region_code': 'first',
    'first_cate_code': 'mean',
    'second_cate_code': 'mean',
    'sales_chan_name': lambda x: (x == 'offline').sum()/len(x),
    'item_price': 'mean',
    'ord_qty': 'sum',
    'season': 'max',
    'season_month': 'max',
    'promotion': 'max',
    'holiday': 'max'
}

df_result_month = df_data_month.groupby(group_cols).agg(agg_dict).reset_index()
df_result_month.insert(0,'order_date',df_data_month['order_date'].dt.to_period('M').dt.to_timestamp())
df_result_month['order_date']=df_result_month['order_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_result_month = df_result_month.rename(columns={
    'item_price': 'mean_item_price',
    'ord_qty': 'sum_ord_qty'
}).sort_values(group_cols + ['sales_region_code'])
df_result_month

,order_date,year,month,item_code,sales_region_code,first_cate_code,second_cate_code,sales_chan_name,mean_item_price,sum_ord_qty,season,season_month,promotion,holiday
0,2015-09-01,2015,9,20003,104,301,405,0.00,739.57,1578,3,3,0,1
1,2015-09-01,2015,9,20006,104,307,403,0.00,542.00,106,3,3,0,1
2,2015-09-01,2015,9,20007,102,305,412,0.74,97.97,7494,3,3,0,1
3,2015-09-01,2015,9,20008,102,303,401,1.00,658.49,1580,3,3,0,1
4,2015-09-01,2015,9,20009,105,306,407,0.23,673.15,1918,3,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,2015-12-01,2018,12,22080,105,306,407,0.00,502.67,36,4,3,1,0
24833,2015-12-01,2018,12,22081,102,303,401,1.00,3235.12,243,4,3,1,1
24834,2015-12-01,2018,12,22082,103,305,412,1.00,2057.50,18,4,3,1,0
24835,2015-12-01,2018,12,22083,105,303,401,0.23,1683.41,2169,4,3,1,1


In [16]:
ywhfun.save(df_result_month, "特征衍生_月.xlsx", type_file="xlsx")